# AI-модуль (LLM-based scoring)

## Цель работы:

Разработать прототип AI-модуля, который оценивает, насколько кандидат подходит под конкретную вакансию,  
и возвращает **строго структурированный, объяснимый и воспроизводимый скоринг**.

Ключевая задача - уйти от:
- поиска по ключевым словам без понимания контекста,
- нестабильных LLM-ответов,
- галлюцинаций (придуманный опыт, навыки, соответствия).

Решение ориентировано на использование в HR-сценариях, где важны:
- аргументация оценки,
- доверие к результату,
- возможность Human-in-the-Loop.


In [4]:
!pip -q install openai datasets pandas numpy scikit-learn jsonschema tiktoken tqdm


В рамках прототипа используются Оpenai - доступ к LLM API

Окружение: Google Colab.

API-ключ передаётся через переменные окружения (Colab Secrets),  
что соответствует базовым требованиям безопасности и production-подходу.




In [24]:
import os
from google.colab import userdata

api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in Colab Secrets")

os.environ["OPENAI_API_KEY"] = api_key



In [29]:
import re
import json
import time
import math
import random

import numpy as np
import pandas as pd
from tqdm import tqdm
from jsonschema import validate, ValidationError
import tiktoken

from openai import OpenAI

client = OpenAI()

MODEL = "gpt-4o-mini"
PROMPT_VERSION = "v1.0"
TEMPERATURE = 0.0


## Строгий ответ (JSON Schema)

Для минимизации галлюцинаций и повышения надёжности:
- финальный ответ LLM должен строго соответствовать JSON Schema,
- при несоответствии выполняется автоматическая попытка repair

Это позволяет:
- безопасно использовать результат в downstream-системах,
- гарантировать стабильный формат для HR и аналитиков.


In [30]:
AI_SCORING_SCHEMA = {
  "type": "object",
  "required": ["prompt_version","final_score","evaluation","missing_info","contradictions","meta"],
  "properties": {
    "prompt_version": {"type":"string"},
    "final_score": {"type":"number", "minimum":0, "maximum":100},
    "evaluation": {
      "type":"array",
      "items":{
        "type":"object",
        "required":["req_id","requirement","priority","status","evidence","reason","score_contribution"],
        "properties":{
          "req_id":{"type":"string"},
          "requirement":{"type":"string"},
          "priority":{"type":"string", "enum":["must","should"]},
          "status":{"type":"string", "enum":["supported","contradicted","missing","unclear"]},
          "evidence":{
            "type":"array",
            "items":{
              "type":"object",
              "required":["quote"],
              "properties":{"quote":{"type":"string"}}
            }
          },
          "reason":{"type":"string"},
          "score_contribution":{"type":"number"}
        }
      }
    },
    "missing_info":{"type":"array", "items":{"type":"string"}},
    "contradictions":{"type":"array", "items":{"type":"string"}},
    "meta":{
      "type":"object",
      "required":["latency_ms","input_tokens","output_tokens","model","n_llm_calls","json_valid","json_repaired"],
      "properties":{
        "latency_ms":{"type":"number"},
        "input_tokens":{"type":"number"},
        "output_tokens":{"type":"number"},
        "model":{"type":"string"},
        "n_llm_calls":{"type":"number"},
        "json_valid":{"type":"number"},
        "json_repaired":{"type":"number"}
      }
    }
  }
}


## Fairness и защита от bias

Перед передачей данных в LLM выполняется минимальная очистка текста:
- удаление email, телефонов, URL,
- маскирование дат рождения и гендерных маркеров.

Это снижает риск:
- дискриминации по полу и возрасту,
- утечки персональных данных,
- смещения модели в сторону нерелевантных признаков.


In [31]:
# --- PII/Fairness sanitization (минимально, но надежно) ---
_email_re = re.compile(r"\b[\w\.-]+@[\w\.-]+\.\w+\b")
_phone_re = re.compile(r"\+?\d[\d\s\-\(\)]{8,}\d")
_url_re   = re.compile(r"https?://\S+")
_birth_re = re.compile(r"\b(\d{1,2}[./-]\d{1,2}[./-]\d{2,4})\b")
_gender_re = re.compile(r"\b(мужск(ой|ая)|женск(ий|ая))\b", re.IGNORECASE)

def sanitize_text(s: str) -> str:
    """
    Удаляю/маскирую потенциальные PII и защищённые признаки.
    Это снижает риск bias и делает пайплайн ближе к production-подходу.
    """
    if not isinstance(s, str):
        return ""
    s = _email_re.sub("[EMAIL]", s)
    s = _phone_re.sub("[PHONE]", s)
    s = _url_re.sub("[URL]", s)
    s = _birth_re.sub("[DATE]", s)
    s = _gender_re.sub("[GENDER]", s)
    return s.strip()

def count_tokens(text: str, model: str = MODEL) -> int:
    try:
        enc = tiktoken.encoding_for_model(model)
    except KeyError:
        enc = tiktoken.get_encoding("o200k_base")
    return len(enc.encode(text or ""))

def chat_json(system: str, user: str, temperature: float = TEMPERATURE):
    """
    Вызов LLM с принудительным возвратом JSON.
    Метрики токенов берём из usage (это основной источник правды).
    """
    t0 = time.time()
    resp = client.chat.completions.create(
        model=MODEL,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        response_format={"type": "json_object"},
    )
    latency_ms = (time.time() - t0) * 1000
    content = resp.choices[0].message.content
    data = json.loads(content)
    in_tok = resp.usage.prompt_tokens
    out_tok = resp.usage.completion_tokens
    return data, latency_ms, in_tok, out_tok


In [32]:
# Загрузка основного датасета.
# Формат файла: train.csv, разделитель '|'
df = pd.read_csv(
    "train.csv",
    sep="|",
    engine="python",
    on_bad_lines="skip"
)

# Проверяем наличие обязательных колонок,
# без которых невозможно корректно сопоставлять резюме и вакансии
required_cols = ["idCv", "idVacancy", "cv_status"]
for c in required_cols:
    if c not in df.columns:
        raise ValueError(f"В датасете отсутствует обязательная колонка: {c}")

# Базовая диагностика датасета
print("Размер датасета:", df.shape)
print("Распределение целевой переменной cv_status:")
print(df["cv_status"].value_counts(dropna=False).head(10))


Размер датасета: (150493, 91)
Распределение целевой переменной cv_status:
cv_status
Отказ          129503
Приглашение     20990
Name: count, dtype: int64


## Формирование текстов резюме и вакансий

Исходные поля датасета имеют сложную структуру  
(строки, JSON, списки, частично заполненные значения).

Для LLM:
- резюме и вакансия собираются в **единый нормализованный текст**,
- используются только потенциально релевантные поля,
- пустые и шумовые значения отбрасываются.

Это позволяет модели работать с чистым, читаемым контекстом.


In [33]:
PII_COLS = {
    "birthday","gender","age","locality","localityName",
    "vacancyAddress","vacancyAddressHouse","vacancyAddressAdditionalInfo",
    "geo","contactPerson","contactSource","company","fullCompanyName",
    "addressCode","addressOffice","contactList"
}

def safe_str(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    s = str(x).strip()
    return "" if s.lower() == "nan" else s

def try_json(x):
    s = safe_str(x)
    if not s:
        return None
    if isinstance(x, (list, dict)):
        return x
    try:
        return json.loads(s)
    except Exception:
        return None

def list_to_text(x, key=None, limit=30):
    """
    Превращаю json-колонки (списки/словари) в компактный текст.
    Это облегчает LLM работу и делает данные единообразными.
    """
    obj = try_json(x)
    if obj is None:
        return safe_str(x)

    if isinstance(obj, list):
        items = []
        for it in obj[:limit]:
            if isinstance(it, dict):
                if key and it.get(key):
                    items.append(str(it[key]))
                else:
                    vals = [str(v) for v in it.values() if v is not None][:2]
                    if vals:
                        items.append(" / ".join(vals))
            else:
                items.append(str(it))
        return ", ".join([i for i in items if i])

    if isinstance(obj, dict):
        pairs = []
        for k, v in list(obj.items())[:limit]:
            if v is None:
                continue
            pairs.append(f"{k}: {v}")
        return "; ".join(pairs)

    return safe_str(x)

def build_resume_text(row) -> str:
    parts = []
    parts.append(f"Желаемая должность: {safe_str(row.get('positionName',''))}")
    parts.append(f"Навыки: {list_to_text(row.get('skills_cv',''))}")
    parts.append(f"Hard skills: {list_to_text(row.get('hardSkills_cv',''))}")
    parts.append(f"Soft skills: {list_to_text(row.get('softSkills_cv',''))}")
    parts.append(f"Опыт (лет): {safe_str(row.get('experience',''))}")
    parts.append(f"Опыт работы: {list_to_text(row.get('workExperienceList',''))}")
    parts.append(f"Образование: {list_to_text(row.get('educationList',''))} {safe_str(row.get('education',''))}".strip())
    parts.append(f"Языки: {list_to_text(row.get('languageKnowledge_cv',''))}")
    parts = [p for p in parts if p.split(": ",1)[-1].strip()]
    return "\n".join(parts)

def build_vacancy_text(row) -> str:
    parts = []
    parts.append(f"Вакансия: {safe_str(row.get('vacancyName',''))}")
    parts.append(f"Квалификации/требования: {safe_str(row.get('qualifications',''))}")
    parts.append(f"Доп. требования: {safe_str(row.get('additionalRequirements',''))}")
    parts.append(f"Требования к позиции: {safe_str(row.get('positionRequirements',''))}")
    parts.append(f"Обязанности: {safe_str(row.get('responsibilities',''))}")
    parts.append(f"Условия: {safe_str(row.get('conditions',''))}")
    parts.append(f"Hard skills: {list_to_text(row.get('hardSkills_vacancy',''))}")
    parts.append(f"Soft skills: {list_to_text(row.get('softSkills_vacancy',''))}")
    parts.append(f"Skills: {list_to_text(row.get('skills_vacancy',''))}")
    parts.append(f"Требуемый опыт: {safe_str(row.get('experienceRequirements',''))}")
    parts.append(f"График: {safe_str(row.get('scheduleType_vacancy',''))}")
    parts.append(f"Требования к образованию: {list_to_text(row.get('educationRequirements',''))}")
    parts = [p for p in parts if p.split(": ",1)[-1].strip()]
    return "\n".join(parts)

df["resume_text"] = df.apply(build_resume_text, axis=1).map(sanitize_text)
df["vacancy_text"] = df.apply(build_vacancy_text, axis=1).map(sanitize_text)

print("empty resume_text rate:", (df["resume_text"].str.len() < 50).mean())
print("empty vacancy_text rate:", (df["vacancy_text"].str.len() < 50).mean())
print("resume_text len stats:\n", df["resume_text"].str.len().describe())
print("vacancy_text len stats:\n", df["vacancy_text"].str.len().describe())


empty resume_text rate: 6.644827334161722e-06
empty vacancy_text rate: 0.0
resume_text len stats:
 count    150493.000000
mean        397.856425
std         399.449194
min          36.000000
25%         189.000000
50%         278.000000
75%         453.000000
max       11667.000000
Name: resume_text, dtype: float64
vacancy_text len stats:
 count    150493.000000
mean       1111.171868
std         842.517205
min         114.000000
25%         558.000000
50%         870.000000
75%        1409.000000
max       25614.000000
Name: vacancy_text, dtype: float64


## LLM-пайплайн и промпт-дизайн

Пайплайн состоит из двух логически разделённых шагов:

1. **Извлечение требований вакансии**
   - только по тексту вакансии,
   - без домысливания,
   - с приоритетами `must / should`.

2. **Скоринг резюме относительно требований**
   - каждый пункт оценивается отдельно,
   - статус возможен только при наличии цитаты,
   - итоговый балл агрегируется объяснимо.


In [34]:
MAX_RESUME_CHARS = 4000
MAX_VACANCY_CHARS = 5000

def clip_text(s: str, limit: int) -> str:
    s = s or ""
    return s[:limit]

SYSTEM_EXTRACT_RU = """Ты извлекаешь требования из текста вакансии строго по тексту.
Запрещено добавлять требования, которых нет в вакансии.
Верни только JSON.
"""

USER_EXTRACT_TMPL_RU = """ТЕКСТ_ВАКАНСИИ:
{vacancy}

Задача:
Выдели ключевые требования к кандидату (только то, что явно указано).
Сфокусируйся на:
- Hard skills / технологии / инструменты
- Опыт (лет, уровень)
- Ограничения (версии, обязательные условия)
- Образование / график (если явно важно)

Верни JSON строго в формате:
{{
  "requirements":[
    {{
      "req_id":"R1",
      "requirement":"кратко и конкретно",
      "priority":"must|should",
      "weight":0-1
    }}
  ]
}}

Правила:
- "must" ставь только если требование явно обязательное ("обязательно", "требуется", "must have").
- Если детали нет — не придумывай.
"""

SYSTEM_SCORE_RU = """Ты модуль скоринга резюме относительно вакансии.

КРИТИЧЕСКИЕ ПРАВИЛА (анти-галлюцинации):
1) Статус "supported" или "contradicted" можно ставить ТОЛЬКО если есть цитата из резюме (evidence.quote).
2) Если цитаты нет — ставь "missing" или "unclear".
3) Игнорируй и НЕ используй защищённые признаки (пол, возраст, имя, национальность, адрес), даже если они присутствуют.
4) Не считай противоречием близкие уровни образования без явного запрета в вакансии (например, "среднее" vs "среднее профессиональное").

Верни только JSON.
"""

USER_SCORE_TMPL_RU = """ТЕКСТ_ВАКАНСИИ:
{vacancy}

ТРЕБОВАНИЯ_JSON:
{req_json}

ТЕКСТ_РЕЗЮМЕ:
{resume}

Верни JSON со следующими полями:
- prompt_version
- final_score (0..100)
- evaluation: по каждому требованию:
  req_id, requirement, priority,
  status (supported/contradicted/missing/unclear),
  evidence: список объектов {{quote}},
  reason (коротко, понятно HR),
  score_contribution (число)
- missing_info: список важных данных, которых не хватает в резюме
- contradictions: список противоречий
- meta: latency_ms/input_tokens/output_tokens/model/n_llm_calls/json_valid/json_repaired (поставь 0 — код перезапишет)

Гайд по скорингу:
- must требования определяют итог сильнее.
- contradicted must => сильный минус.
- missing must => минус, но мягче.
"""

SYSTEM_REPAIR_RU = """Исправь JSON так, чтобы он строго соответствовал JSON Schema.
Нельзя добавлять новую информацию — только сделать JSON валидным.
Верни только JSON.
"""


## Основная функция ai_scoring

Функция ai_scoring:
- принимает текст вакансии и резюме,
- возвращает структурированный JSON-скоринг,
- инкапсулирует весь LLM-пайплайн.

Внутри:
- кэширование требований вакансии,
- контроль токенов и latency,
- строгая JSON-валидация,
- автоматический repair при необходимости.

Функция является центральным интерфейсом модуля.


In [35]:
_req_cache = {}

def enforce_no_hallucination(out: dict) -> dict:
    """
    Code-level safeguard:
    supported/contradicted допустимы только при наличии evidence.quote.
    Это защищает от галлюцинаций даже при ошибке модели.
    """
    for item in out.get("evaluation", []):
        status = item.get("status")
        evidence = item.get("evidence") or []
        has_quote = any(isinstance(e, dict) and (e.get("quote") or "").strip() for e in evidence)
        if status in {"supported", "contradicted"} and not has_quote:
            item["status"] = "unclear"
            item["score_contribution"] = 0
            item["reason"] = (item.get("reason","").strip() + " (нет подтверждающей цитаты в резюме)").strip()
            item["evidence"] = []
    return out

def ai_scoring(vacancy_text: str, resume_text: str) -> dict:
    """
    Main API function for the test:
    input: raw vacancy_text + resume_text
    output: strict JSON scoring according to schema
    """
    vacancy = clip_text(sanitize_text(vacancy_text), MAX_VACANCY_CHARS)
    resume  = clip_text(sanitize_text(resume_text),  MAX_RESUME_CHARS)

    total_latency = 0.0
    total_in = 0
    total_out = 0
    n_calls = 0

    vkey = hash(vacancy)

    # 1) Extract requirements (cached per vacancy text)
    if vkey not in _req_cache:
        req_data, lat, tin, tout = chat_json(
            SYSTEM_EXTRACT_RU,
            USER_EXTRACT_TMPL_RU.format(vacancy=vacancy),
            temperature=0.0
        )
        _req_cache[vkey] = req_data
        total_latency += lat; total_in += tin; total_out += tout; n_calls += 1

    req_data = _req_cache[vkey]
    req_json = json.dumps(req_data, ensure_ascii=False)

    # 2) Score candidate vs extracted requirements
    out_data, lat, tin, tout = chat_json(
        SYSTEM_SCORE_RU,
        USER_SCORE_TMPL_RU.format(vacancy=vacancy, req_json=req_json, resume=resume),
        temperature=0.0
    )
    total_latency += lat; total_in += tin; total_out += tout; n_calls += 1

    # 3) Enforce anti-hallucination on code level
    out_data = enforce_no_hallucination(out_data)

    # 4) Fill meta + validate/repair
    out_data["prompt_version"] = PROMPT_VERSION
    out_data["meta"] = {
        "latency_ms": float(total_latency),
        "input_tokens": float(total_in),
        "output_tokens": float(total_out),
        "model": MODEL,
        "n_llm_calls": float(n_calls),
        "json_valid": 1.0,
        "json_repaired": 0.0
    }

    try:
        validate(instance=out_data, schema=AI_SCORING_SCHEMA)
        return out_data
    except ValidationError:
        # one repair attempt
        repair_user = (
            "INVALID_JSON:\n" + json.dumps(out_data, ensure_ascii=False) +
            "\n\nSCHEMA:\n" + json.dumps(AI_SCORING_SCHEMA, ensure_ascii=False)
        )
        fixed, lat2, tin2, tout2 = chat_json(SYSTEM_REPAIR_RU, repair_user, temperature=0.0)
        total_latency += lat2; total_in += tin2; total_out += tout2; n_calls += 1

        fixed = enforce_no_hallucination(fixed)

        fixed["prompt_version"] = PROMPT_VERSION
        fixed["meta"] = {
            "latency_ms": float(total_latency),
            "input_tokens": float(total_in),
            "output_tokens": float(total_out),
            "model": MODEL,
            "n_llm_calls": float(n_calls),
            "json_valid": 0.0,
            "json_repaired": 1.0
        }
        validate(instance=fixed, schema=AI_SCORING_SCHEMA)
        return fixed


In [37]:
# test (одна пара из датасета)
row = df.dropna(subset=["vacancy_text","resume_text"]).iloc[0]
res = ai_scoring(row["vacancy_text"], row["resume_text"])
print("final_score:", res["final_score"])
print("n_eval:", len(res["evaluation"]))
print(json.dumps(res, ensure_ascii=False, indent=2)[:1500])


final_score: 25
n_eval: 4
{
  "prompt_version": "v1.0",
  "final_score": 25,
  "evaluation": [
    {
      "req_id": "R1",
      "requirement": "Наличие опыта работы",
      "priority": "should",
      "status": "supported",
      "evidence": [
        {
          "quote": "Опыт (лет): 2"
        }
      ],
      "reason": "Кандидат имеет 2 года опыта работы.",
      "score_contribution": 50
    },
    {
      "req_id": "R2",
      "requirement": "Среднее профессиональное образование",
      "priority": "should",
      "status": "contradicted",
      "evidence": [
        {
          "quote": "Образование: 2015 / ТСОШ Среднее"
        }
      ],
      "reason": "Кандидат имеет среднее общее образование, а требуется среднее профессиональное.",
      "score_contribution": -50
    },
    {
      "req_id": "R3",
      "requirement": "Ответственность",
      "priority": "should",
      "status": "missing",
      "evidence": [],
      "reason": "Нет информации о наличии ответственности.",
  

## Массовый прогон и сбор метрик

Для оценки качества решения:
- используется 100 пар (резюме, вакансия),
- не менее 10 уникальных вакансий,
- каждый скоринг выполняется независимо.

Собираются:
- итоговые оценки,
- latency,
- входные и выходные токены,
- стабильность JSON-ответов.


In [38]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

np.random.seed(42)

pairs_df = (
    df.dropna(subset=["resume_text","vacancy_text","cv_status","idVacancy","idCv"])
      .sample(n=100, random_state=42)
      .copy()
)

print("pairs:", pairs_df.shape, "unique vacancies:", pairs_df["idVacancy"].nunique())

rows_out = []
for _, row in tqdm(pairs_df.iterrows(), total=len(pairs_df), desc="scoring_100"):
    out = ai_scoring(row["vacancy_text"], row["resume_text"])
    rows_out.append({
        "idVacancy": row["idVacancy"],
        "idCv": row["idCv"],
        "cv_status": row["cv_status"],
        "final_score": out["final_score"],
        "latency_ms": out["meta"]["latency_ms"],
        "input_tokens": out["meta"]["input_tokens"],
        "output_tokens": out["meta"]["output_tokens"],
        "json_valid": out["meta"]["json_valid"],
        "json_repaired": out["meta"]["json_repaired"],
        "n_requirements": len(out.get("evaluation", [])),
    })

results = pd.DataFrame(rows_out)
results.head()


pairs: (100, 93) unique vacancies: 100


scoring_100: 100%|██████████| 100/100 [34:37<00:00, 20.77s/it]


,idVacancy,idCv,cv_status,final_score,latency_ms,input_tokens,output_tokens,json_valid,json_repaired,n_requirements
0,8a4f1ec5-9005-11ec-a69c-4febb26dc4ec,6807cc00-2703-11ed-81e0-fdf9f86d256a,Отказ,10,28034.849882,3653.0,1589.0,1.0,0.0,10
1,10d6da82-9280-11ef-b4d5-e7d0d2cf29b1,aeba9940-f875-11ed-a410-715f91579781,Отказ,50,9257.095814,1430.0,379.0,1.0,0.0,2
2,f31ec640-1b09-11ee-9f4d-9586bb63c653,e0db3e20-a0ee-11ef-b469-c7bd3a8c3ec7,Отказ,30,18086.399078,1243.0,779.0,1.0,0.0,5
3,7dbbed98-91de-11ef-8716-cb26dff57dd7,c76d2ca0-50a9-11ee-b447-d53cd48f29e6,Отказ,0,17188.035011,1380.0,884.0,1.0,0.0,6
4,1d275ea5-fdda-11ec-b057-57fc951f3846,c6d707b0-fb4c-11ec-bcb3-839f0d9a4379,Отказ,60,22299.293995,2432.0,1040.0,1.0,0.0,7


## Оценка качества скоринга

В качестве ground truth используется:
- cv_status из датасета,
- приглашение → 1, отказ → 0.

Рассчитываются:
- Accuracy,
- Precision,
- Recall,
- F1-score

Метрики считаются для разных порогов скоринга  
для анализа trade-off между precision и recall.


In [40]:
y_true = results["cv_status"].astype(str).str.contains("Приглаш", case=False, na=False).astype(int)

def metrics_at_threshold(th: int):
    y_pred = (results["final_score"] >= th).astype(int)
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    return acc, prec, rec, f1

for th in [50, 60, 70]:
    acc, prec, rec, f1 = metrics_at_threshold(th)
    print(f"TH={th} | acc={acc:.3f} | prec={prec:.3f} | rec={rec:.3f} | f1={f1:.3f}")

# pipeline stats
def mean_std(x):
    return float(np.mean(x)), float(np.std(x, ddof=1))

print("\nPIPELINE STATS (mean/std)")
print("latency_ms:", mean_std(results["latency_ms"]))
print("input_tokens:", mean_std(results["input_tokens"]))
print("output_tokens:", mean_std(results["output_tokens"]))

# stability stats
json_valid_rate = float(results["json_valid"].mean())
repair_rate = float(results["json_repaired"].mean())
avg_nreq = float(results["n_requirements"].mean())

print("\nSTABILITY / QUALITY")
print("json_valid_rate:", json_valid_rate)
print("repair_rate:", repair_rate)
print("avg_n_requirements:", avg_nreq)
print("class_balance:", y_true.value_counts().to_dict())


TH=50 | acc=0.560 | prec=0.051 | rec=0.222 | f1=0.083
TH=60 | acc=0.670 | prec=0.071 | rec=0.222 | f1=0.108
TH=70 | acc=0.850 | prec=0.200 | rec=0.222 | f1=0.211

PIPELINE STATS (mean/std)
latency_ms: (20751.80416584015, 10832.756939795698)
input_tokens: (1673.41, 623.5318610281047)
output_tokens: (944.05, 455.9294286832051)

STABILITY / QUALITY
json_valid_rate: 1.0
repair_rate: 0.0
avg_n_requirements: 6.29
class_balance: {0: 91, 1: 9}


Модель демонстрирует ожидаемое поведение при сильном дисбалансе классов (9% приглашений):

При увеличении порога скоринга (TH=50 → TH=70) растёт precision и F1, что говорит о более строгом и аккуратном отборе кандидатов.

Recall остаётся стабильным (~0.22).

Accuracy существенно увеличивается при высоких порогах, что ожидаемо при доминирующем классе отказов и не используется как основная метрика.

Оптимальным порогом для сценария предварительного отбора кандидатов является диапазон TH ≈ 60–70, где достигается баланс между точностью и консервативностью скоринга.

In [41]:
results.to_csv(f"results_{PROMPT_VERSION}.csv", index=False)
pairs_df[["idCv","idVacancy","cv_status"]].to_csv(f"pairs_{PROMPT_VERSION}.csv", index=False)
print("saved:", f"results_{PROMPT_VERSION}.csv", f"pairs_{PROMPT_VERSION}.csv")


saved: results_v1.0.csv pairs_v1.0.csv


## Калибровка порога принятия решения

Так как скоринг непрерывен (0–100),
подбирается оптимальный порог по метрике F1.

Это демонстрирует:
- осознанный подход к принятию решений,
- возможность адаптации под разные бизнес-сценарии
  (массовый найм vs точечный подбор).


In [42]:
best = None
for th in range(0, 101, 5):
    acc, prec, rec, f1 = metrics_at_threshold(th)
    if best is None or f1 > best[-1]:
        best = (th, acc, prec, rec, f1)

print("\nBEST THRESHOLD (by F1):")
print(f"TH={best[0]} | acc={best[1]:.3f} | prec={best[2]:.3f} | rec={best[3]:.3f} | f1={best[4]:.3f}")



BEST THRESHOLD (by F1):
TH=35 | acc=0.550 | prec=0.125 | rec=0.667 | f1=0.211


В ходе экспериментов были рассмотрены разные пороги принятия решения, соответствующие разным HR-сценариям.

TH ≈ 60–70
Как мы видим выше, при данных порогах достигается более консервативный режим скоринга:

выше precision,

ниже количество ложных приглашений.

Такой диапазон порогов оптимален для сценария предварительного shortlisting’а, когда AI используется как фильтр для отсечения явно неподходящих кандидатов.

TH = 35 (оптимум по F1-score)
При калибровке по F1-score оптимальным оказался порог TH = 35, обеспечивающий:

высокий recall (0.667),

максимальный F1-score (0.211) в условиях сильного дисбаланса классов.

Данный порог подходит для сценария recall-first, где важно не упустить потенциально релевантных кандидатов, а финальное решение принимается рекрутером.

## Версионность и воспроизводимость промптов

Все используемые промпты:
- сохраняются в реестр,
- хэшируются,
- связываются с версией и моделью.

Это позволяет:
- отслеживать изменения,
- воспроизводить эксперименты,
- безопасно развивать систему дальше.


In [43]:
import hashlib, json

PROMPTS = {
    "SYSTEM_EXTRACT_RU": SYSTEM_EXTRACT_RU,
    "USER_EXTRACT_TMPL_RU": USER_EXTRACT_TMPL_RU,
    "SYSTEM_SCORE_RU": SYSTEM_SCORE_RU,
    "USER_SCORE_TMPL_RU": USER_SCORE_TMPL_RU,
    "SYSTEM_REPAIR_RU": SYSTEM_REPAIR_RU,
}

def sha12(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()[:12]

prompt_registry = {
    "prompt_version": PROMPT_VERSION,
    "model": MODEL,
    "hashes": {k: sha12(v) for k, v in PROMPTS.items()},
}

with open(f"prompt_registry_{PROMPT_VERSION}.json", "w", encoding="utf-8") as f:
    json.dump(prompt_registry, f, ensure_ascii=False, indent=2)

print(prompt_registry)



{'prompt_version': 'v1.0', 'model': 'gpt-4o-mini', 'hashes': {'SYSTEM_EXTRACT_RU': '6fe84c21547b', 'USER_EXTRACT_TMPL_RU': '007ea1c1019d', 'SYSTEM_SCORE_RU': '2679b16ac5c5', 'USER_SCORE_TMPL_RU': '8b438d3dda4a', 'SYSTEM_REPAIR_RU': '748c6504179d'}}


Пайплайн продемонстрировал устойчивую работу на выборке из 100 пар «вакансия–резюме» с реальными HR-данными и сильным дисбалансом классов (≈9% приглашений).

По результатам оценки:

при росте порога скоринга увеличивается точность и снижается количество ложных приглашений;

при снижении порога значительно растёт recall, что важно для сценариев первичного отбора.

Калибровка показала, что:

TH ≈ 35 оптимален для recall-oriented сценария (поиск максимума потенциально подходящих кандидатов);

TH ≈ 60–70 подходит для консервативного shortlisting’а, когда приоритет — качество приглашений.

Таким образом, итоговый скоринг и порог отсечения являются бизнес-настраиваемыми параметрами, а не фиксированными константами, и могут адаптироваться под конкретный HR-процесс без изменения логики модели.

Система обеспечивает воспроизводимый, объяснимый и формально валидный результат, что делает её пригодной для использования как базового AI-ядра в задачах автоматизированного подбора персонала.

## В дальнейшем решение может быть усилено за счёт:

более глубокой и структурированной предобработки данных (нормализация опыта, образования, навыков, унификация форматов списков и JSON-полей);

выделения и типизации ключевых сущностей (навыки, технологии, уровни опыта) до этапа LLM-скоринга;

использования предварительного фильтра (rule-based или embedding-based) для сокращения числа LLM-вызовов;

более точной калибровки весов требований (must/should) под конкретные отрасли и типы вакансий;

расширения Human-in-the-Loop сценариев для разметки пограничных случаев и дообучения логики скоринга.